In [ ]:

%pip install python-dotenv

## 环境初始化

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
# print(os.environ)

# os.environ["OPENAI_API_TYPE"] = os.environ.get('OPENAI_API_TYPE')
# os.environ["OPENAI_API_BASE"] = os.environ.get('OPENAI_API_BASE')
# os.environ["OPENAI_API_KEY"] = os.environ.get('OPENAI_API_KEY')
# os.environ["OPENAI_API_VERSION"] = os.environ.get('OPENAI_API_VERSION')


AI_TEMPERATURE=0.0
AI_MAX_TOKENS=2048

In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI

def load_azureLLM():
    # chat
    chat = AzureChatOpenAI(
        deployment_name=os.environ.get('DEPLOYMENT_NAME_CHAT'),
        temperature=AI_TEMPERATURE,
        max_tokens=AI_MAX_TOKENS,
        )
    # llm
    llm = AzureOpenAI(
        deployment_name=os.environ.get('DEPLOYMENT_NAME_LLM'),
        model_name="text-davinci-003",
        temperature=AI_TEMPERATURE,
        max_tokens=AI_MAX_TOKENS,
        )
    
    return llm, chat

llm, chat = load_azureLLM()
# print(llm)
# print(chat)
# llm("Tell me a joke")

## 营养建议

### 提示模板

In [70]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

fitness_plan = ResponseSchema(name="fitness_plan", description="训练计划输出json格式")
nutrition_plan =ResponseSchema(name="nutrition_plan", description="营养计划输出json格式")

response_schemas = [fitness_plan, nutrition_plan]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"fitness_plan": string  // 训练计划输出json格式
	"nutrition_plan": string  // 营养计划输出json格式
}
```


### 提示信息

In [71]:


goal = "减脂瘦身"
fitness = """{
    "frequency": "3",
    "date": "周一、周三、周六",
    "time": "晚上",
    "equipment": "瑜伽垫",
    "music": "舒缓类",
    "exclude": "跑跳类动作",
    "duration": "不超过20分钟"
}"""
person = """{
    "gender": "女",
    "age": "20岁",
    "height": "150cm",
    "weight": "50kg"
}"""
health = """{
    "health": "良好",
    "drug": "无",
    "allergy": "牛奶",
}"""
foods = """{
    "frequency": "2",
    "favorite ": "香蕉",
    "dislike ": "青椒",
    "allergy": "牛奶"
    "snacks": "50kg"
}"""

user_prompt_template = """你将作为一位备受赞誉的健康与营养专家 FitnessGPT。\
    我希望你能根据我提供的个人信息、健康状态、健身目标、运动偏好及饮食偏好，为我定制一套个性化的饮食和运动计划。\
    我需要你为我详细制定运动计划。需要列出每次运动课程的动作清单。课程动作需要包括运动时间或轮次细节。\
    我也需要你帮我详细规划我的饮食计划，并列出一份详细的饮食清单，清单上需要包括每种食品的数量及饮食方式。\
    请避免任何不必要的描述性文本，以及解释性描述，只输出规划设计的计划内容。不论在什么情况下，都请保持角色设定不变。\
    Let's think step by step.\
    
    我的个人信息: ```{private_info}```。\
    我的健康状态是: ```{health_info}```。\
    我的健身目标是: ```{fitness_goal}```。\
    我的运动偏好是:```{fitness_preference}```。\
    我的饮食偏好是: ```{fitness_foods}```。

    {format_instructions}
    """


### 分类Chat消息
> ChatPromptTemplate

In [72]:
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

ft_prompt = PromptTemplate(
    template=user_prompt_template,
    input_variables=["private_info","health_info","fitness_goal","fitness_preference","fitness_foods", "format_instructions"],
)
human_prompt = HumanMessagePromptTemplate(prompt=ft_prompt)

# 组合prompt. System/AI/Human
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

# 格式化prompt
human_message = chat_prompt.format_prompt(
    private_info=person,
    health_info=health,
    fitness_goal=goal,
    fitness_preference=fitness,
    fitness_foods=foods,
    format_instructions=format_instructions,
).to_messages()
# print(chat_message)

ai_message = chat.predict_messages(human_message)

In [74]:
# print(ai_message.content)
ai_out_dict = output_parser.parse(ai_message.content)
ai_out_dict

{'fitness_plan': {'Monday': {'time': '晚上',
   'duration': '不超过20分钟',
   'equipment': '瑜伽垫',
   'music': '舒缓类',
   'exclude': '跑跳类动作',
   'exercises': ['下犬式', '平衡式', '三角式', '侧弯式', '蝴蝶式']},
  'Wednesday': {'time': '晚上',
   'duration': '不超过20分钟',
   'equipment': '瑜伽垫',
   'music': '舒缓类',
   'exclude': '跑跳类动作',
   'exercises': ['猫式', '树式', '战士式', '鸽子式', '蛇式']},
  'Saturday': {'time': '晚上',
   'duration': '不超过20分钟',
   'equipment': '瑜伽垫',
   'music': '舒缓类',
   'exclude': '跑跳类动作',
   'exercises': ['鱼式', '骆驼式', '蝎子式', '鸟式', '蛙式']}},
 'nutrition_plan': {'Monday': {'breakfast': ['燕麦片 50g', '牛奶 200ml', '香蕉 1个'],
   'lunch': ['鸡胸肉 100g', '西兰花 100g', '米饭 50g'],
   'dinner': ['鲈鱼 100g', '蔬菜沙拉 100g', '面包 1片'],
   'snack': ['苹果 1个', '坚果 20g']},
  'Wednesday': {'breakfast': ['全麦面包 1片', '鸡蛋 1个', '牛奶 200ml'],
   'lunch': ['瘦牛肉 100g', '蔬菜沙拉 100g', '米饭 50g'],
   'dinner': ['鸡胸肉 100g', '蔬菜沙拉 100g', '面包 1片'],
   'snack': ['香蕉 1个', '坚果 20g']},
  'Saturday': {'breakfast': ['燕麦片 50g', '牛奶 200ml', '香蕉 1个'],
   

### 抽象Chat消息
> ChatMessagePromptTemplate

In [ ]:
from langchain.prompts import (
    ChatMessagePromptTemplate,
)

chat_message_prompt = ChatMessagePromptTemplate.from_template(role="user",template=user_prompt_template)
chat_message = chat_message_prompt.format_messages(
    private_info=person,
    health_info=health,
    fitness_goal=goal,
    fitness_preference=fitness,
    fitness_foods=foods,
    format_instructions=format_instructions,
)
# print(chat_message)

message = chat.predict_messages(chat_message)

In [ ]:
print(message.content)

In [69]:
out_dict = output_parser.parse(message.content)
out_dict

{'fitness_plan': {'Monday': {'time': '晚上',
   'duration': '不超过20分钟',
   'equipment': '瑜伽垫',
   'music': '舒缓类',
   'exclude': '跑跳类动作',
   'exercises': ['下犬式', '平衡式', '三角式', '蝴蝶式', '猫式', '树式']},
  'Wednesday': {'time': '晚上',
   'duration': '不超过20分钟',
   'equipment': '瑜伽垫',
   'music': '舒缓类',
   'exclude': '跑跳类动作',
   'exercises': ['下犬式', '平衡式', '三角式', '蝴蝶式', '猫式', '树式']},
  'Saturday': {'time': '晚上',
   'duration': '不超过20分钟',
   'equipment': '瑜伽垫',
   'music': '舒缓类',
   'exclude': '跑跳类动作',
   'exercises': ['下犬式', '平衡式', '三角式', '蝴蝶式', '猫式', '树式']}},
 'nutrition_plan': {'Monday': {'breakfast': [{'name': '燕麦片',
     'quantity': '50g'},
    {'name': '牛奶', 'quantity': '100ml', 'exclude': '牛奶'},
    {'name': '香蕉', 'quantity': '1个'}],
   'lunch': [{'name': '鸡胸肉', 'quantity': '100g'},
    {'name': '西兰花', 'quantity': '100g'},
    {'name': '米饭', 'quantity': '50g'}],
   'dinner': [{'name': '鱼肉', 'quantity': '100g'},
    {'name': '蔬菜沙拉', 'quantity': '100g', 'exclude': '青椒'},
    {'name': '面包', 'quan

## 评估动作

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_evaluate_action_standards = """评估维度：\
1. 力量：指肌肉在运动中产生的最大力量。常见的力量训练包括举重、俯卧撑、深蹲等。通过计次评估。\
2. 速度：指肌肉完成一定距离或动作所需时间的能力。常见的速度训练包括快速小跑、跳绳等。通过计时评估。\
3. 耐力：指肌肉长时间持续工作的能力。常见的耐力训练包括深蹲跳、高抬腿等。通过计次或计时评估。\
4. 灵敏：指肌肉对刺激的快速反应能力。常见的灵敏度训练包括跳绳、跳跃、击剑等。通过计次评估。\
5. 协调：指身体各部位肌肉协调运动的能力，常见的协调性训练包括瑜伽、桑巴、舞蹈等。通过计时评估。\
6. 稳定：指骨骼及肌肉在身体静止或运动状态下维持平衡的能力。常见的稳定性训练包括平板支撑、单脚站立等。通过计时评估。\
评估方法：\
根据动作的计次和计时数据，严格评估动作在每个维度的得分。\
评估得分，最低0分，最高10分。如果无法评估，则为0分。
"""

person_info = """{
    "gender": "女",
    "age": "20岁",
    "height": "150cm",
    "weight": "50kg"
}"""
fitness_data = """{
    "action": "深蹲",
    "timing": "60秒",
    "count_type": "次数",
    "count_result": "30次"
}"""

prompt_evaluate_action = """
您是资深的健身运动专家，请严格以``` {standards} ```为评估标准，对用户的运动数据进行评估。\
请仅对用户的运动数据进行评估，不要额外评价。\
严格以json格式化输出下列字段:\
dimension:
    score:
    reason:

Let's think step by step.\

运动数据：### {data} ###
"""


evaluate_template = ChatPromptTemplate.from_template(prompt_evaluate_action)

evaluate_messages = evaluate_template.format_messages(
                    standards=prompt_evaluate_action_standards,
                    data=fitness_data,
                    )

action_evaluate = chat(evaluate_messages)
print(action_evaluate.content)

## 动作升降阶

## 修正课程

## 课程生成